Импортируем необходимые библиотеки.

In [110]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.linear_model import SGDClassifier, RidgeCV, Lasso, ElasticNetCV, MultiTaskLasso, LassoLarsCV, LogisticRegression, PassiveAggressiveClassifier
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split, cross_val_score
import sklearn.preprocessing
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer

import math
import numpy as np
import pandas as pd

Открываем обучающую и тестовую выборки.

In [112]:
train = pd.read_csv("linear_train.txt", header = None, encoding = 'utf8')
test = pd.read_csv("linear_test.txt", names = ['words'], encoding = 'utf8')
sample_submission = pd.read_csv("linear_ans_example.txt", encoding = 'utf8')

In [105]:
train.head()

,0,1
0,﻿Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


In [50]:
X = train[0]
y = train[1]

Функции для проверки слова на наличие первой большой буквы и на наличие всех больших букв. В последствии оказалось, что это даже немного ухудшает результат. (Вероятно благодаря наличию в обучающей выборке фамилий-выбросов типа "ящик")

In [106]:
def f_is_up(x):
    if (x[0].isupper()):
        return 1.0
    else:
        return 0.0

In [107]:
def w_is_up(x):
    if (x.isupper()):
        return 1.0
    else:
        return 0.0

In [108]:
F = []
W = []
for w in X:
    F.append(f_is_up(w))
    W.append(w_is_up(w))

Методом проб и ошибок понимаем, что таргет должен быть записан в виде вероятностей (т.к. скоринг ROC-AUC).

Также в процессе экспериментов был найден алгоритм, лучше всех показывающий себя на кросс-валидации трейновой выборки.

In [111]:
class proba_XGB(xgb.XGBClassifier):
    def predict(self, X):
        return xgb.XGBClassifier.predict_proba(self, X)

In [82]:
pipeline = make_pipeline(CountVectorizer(analyzer = 'char_wb', ngram_range = [3, 3], lowercase = False, max_features = 1000), 
                         TfidfTransformer(norm=u'l1', use_idf=True, smooth_idf=False, sublinear_tf=True),
                         proba_XGB(max_depth=100, learning_rate=0.001, n_estimators=1000, seed = 1337, nthread = 4))
arr = cross_val_score(pipeline, X, y, cv=5, scoring='roc_auc')
print arr
print np.mean(arr)

[ 0.78957077  0.71752906  0.72246378  0.74815822  0.77528784]
0.750601933665


In [42]:
pipeline.fit(X, y)

Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,...   penalty='l1', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [98]:
predictions = pipeline.predict(test['words'])

Однако, его потолок на kaggle - 0.82.

Тогда пробуем логистическую регрессию на больших n-граммах.

In [114]:
class proba_LR(LogisticRegression):
    def predict(self, X):
        return LogisticRegression.predict_proba(self, X)

In [115]:
pipeline = make_pipeline(CountVectorizer(analyzer = 'char_wb', ngram_range = [2, 8], binary = True), 
                         proba_LR(random_state=92))
arr = cross_val_score(pipeline, X, y, cv=5, scoring='roc_auc')
print arr
print np.mean(arr)

[ 0.62755952  0.40237198  0.34451371  0.78375555  0.78321243]
0.58828263804


In [117]:
pipeline.fit(X, y)

Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer='char_wb', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=[2, 8], preprocessor=None, stop_words=None...  penalty='l2', random_state=92, solver='liblinear', tol=0.0001,
     verbose=0, warm_start=False))])

In [118]:
predictions = pipeline.predict(test['words'])

In [119]:
df = pd.DataFrame(data=predictions[:,1], columns=['Answer'])

In [120]:
df.to_csv('please_submission.csv', index=True, index_label='Id')

Итог: 0.92